In [1]:
!unzip "/content/drive/MyDrive/Shop DataSet.zip" -d "/content"

Archive:  /content/drive/MyDrive/Shop DataSet.zip
   creating: /content/Shop DataSet/non shop lifters/
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_0.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_0_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_1_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_10.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_10_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_100.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_100_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_101.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_101_1.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lifter_n_102.mp4  
  inflating: /content/Shop DataSet/non shop lifters/shop_lift

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import zipfile
import numpy as np
import cv2
import random
from sklearn import metrics
from sklearn.classifier_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.animation as animation
from tensorflow.keras.layers import *
from tensorflow.keras.classifiers import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_classifier
import datetime as dt
from sklearn.metrics import performance_score_score, precision_score, recall_score, f1_score
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": False})


In [2]:
DATASET_DIR = "/content/Shop DataSet"
shop_lifters_dir = os.path.join(DATASET_DIR, "shop lifters")
non_shop_lifters_dir = os.path.join(DATASET_DIR, "non shop lifters")

#print the number of video files in each directory
print(f"Found {len(os.listdir(shop_lifters_dir))} video files in the directory: {shop_lifters_dir}")
print(f"Found {len(os.listdir(non_shop_lifters_dir))} video files in the directory: {non_shop_lifters_dir}")

Found 324 video files in the directory: /content/Shop DataSet/shop lifters
Found 531 video files in the directory: /content/Shop DataSet/non shop lifters


In [3]:
!pip install datasets transformers[torch]

In [4]:
from transformers import AutoImageProcessor, AutoModelForVideoClassification, TrainingArguments, Trainer
import torch
from datasets import load_dataset, Features, ClassLabel, Array3D, Value
import numpy as np
import os
import cv2

In [1]:
pip install tensorflow opencv-python numpy tqdm scikit-learn


In [10]:
import cv2
import os
import numpy as np
from tqdm import tqdm

# Set parameters
FRAME_RATE = 25  # Extract 25 frames per second

def extract_frames(video_path, output_folder):
    """Extract frames from a video at specified frame rate."""
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % (fps // FRAME_RATE) == 0:
            frame_filename = os.path.join(output_folder, f"{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
        frame_count += 1

    cap.release()

def process_videos(video_folder, output_folder, dataset_dir): # Added dataset_dir as an argument
    """Process all videos in a given folder."""
    for video in os.listdir(dataset_dir): # Use the passed dataset_dir
        video_path = os.path.join(video_folder, video)
        class_folder = os.path.join(output_folder, os.path.splitext(video)[0])
        extract_frames(video_path, class_folder)

# Set paths
video_data_path = "shoplifting_dataset"  # Your dataset path
output_frames_path = "extracted_frames"
DATASET_DIR = "/content/Shop DataSet" # Define DATASET_DIR here

# Process videos, passing DATASET_DIR to process_videos
process_videos(os.path.join(video_data_path, "shoplifters"), os.path.join(output_frames_path, "shoplifters"), DATASET_DIR)
process_videos(os.path.join(video_data_path, "non_shoplifters"), os.path.join(output_frames_path, "non_shoplifters"), DATASET_DIR)

In [11]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import preprocess_input

# Load Pretrained CNN Model
cnn_classifier = EfficientNetB0(weights="imagenet", include_top=False, pooling="avg")

def extract_features(img_path):
    """Extract deep learning features from an image using EfficientNetB0."""
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    return cnn_classifier.predict(img_array).flatten()

def process_frame_folder(folder_path):
    """Extract features for all images in a folder."""
    features = []
    for img_file in sorted(os.listdir(folder_path)):  # Ensure consistent order
        img_path = os.path.join(folder_path, img_file)
        features.append(extract_features(img_path))
    return np.array(features)

# Process dataset
def extract_dataset_features(base_folder):
    """Extract features for all videos (each as a sequence)."""
    X, y = [], []
    for label, class_name in enumerate(["non_shoplifters", "shoplifters"]):
        class_folder = os.path.join(base_folder, class_name)
        for video_folder in os.listdir(class_folder):
            video_path = os.path.join(class_folder, video_folder)
            frame_features = process_frame_folder(video_path)
            if frame_features.shape[0] > 0:
                X.append(frame_features)  # Each video is a sequence of features
                y.append(label)

    return X, y

X, y = extract_dataset_features(output_frames_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.classifier_selection import train_test_split

# Pad sequences to ensure equal length
max_frames = max([len(seq) for seq in X])  # Find longest sequence
X_padded = pad_sequences(X, maxlen=max_frames, dtype="float32", padding="post", truncating="post")

# Convert labels to NumPy array
y = np.array(y)

# Split dataset
features_train, features_test, labels_train, labels_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [13]:
from tensorflow.keras.classifiers import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking

# Define LSTM Model
classifier = Sequential([
    Masking(mask_value=0.0, input_shape=(max_frames, 1280)),  # Ignore padded values
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile Model
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["performance_score"])

# Train Model
classifier.fit(features_train, labels_train, epochs=20, batch_size=8, validation_data=(features_test, labels_test))


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.2500 - loss: 0.7283 - val_accuracy: 1.0000 - val_loss: 0.2810
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.2544 - val_accuracy: 1.0000 - val_loss: 0.1464
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1531 - val_accuracy: 1.0000 - val_loss: 0.0910
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1021 - val_accuracy: 1.0000 - val_loss: 0.0577
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0534 - val_accuracy: 1.0000 - val_loss: 0.0388
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0481 - val_accuracy: 1.0000 - val_loss: 0.0272
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0443 - val_accuracy: 1.0000 - val_loss: 0.0193
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.0298 - val_accuracy: 1.0000 - val_loss: 0.0139
Epoch 9/20
1/1 ━━━━━━━━━━━━

In [14]:
# Evaluate on test set
loss, performance_score = classifier.evaluate(features_test, labels_test)
print(f"Test Accuracy: {performance_score:.2f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step - accuracy: 1.0000 - loss: 0.0010
Test Accuracy: 1.00
